In [79]:
%load_ext autoreload
%autoreload 2

import os
os.environ['TORCH_HOME'] = "/disks/sdb/torch_home"

import logging
logging.getLogger().setLevel(logging.WARNING)

from awesome_glue.task import *
from awesome_glue.config import Config

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [87]:
config = Config()
config.task_id = 'AGNEWS'
task = Task(config)
task.from_pretrained()
config

values = list(task.vocab.get_index_to_token_vocabulary().values())
results = []
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    result = [ele['probs'][0] for ele in result]
    results.extend(result)
pairs = sorted(zip(values, results), key=lambda x: x[1], reverse=False)

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 18.8 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.0567 seconds


In [90]:
config._model_name = 'AGNEWS-lstm-hot.1.5.con'
task = Task(config)
task.from_pretrained()
task.model.eval()
task.evaluate_model()
config

values = list(task.vocab.get_index_to_token_vocabulary().values())
results = []
from allennlp.common.util import lazy_groups_of
for group in lazy_groups_of(values, 1024):
    result = task.predictor.predict_batch_json([{"sent": ele} for ele in group])
    result = [ele['probs'][0] for ele in result]
    results.extend(result)
pairs2 = sorted(zip(values, results), key=lambda x: x[1], reverse=False)

cache for AGNEWS-spacy.data exists
*** load AGNEWS-spacy.data from cache cost 17.6 seconds
cache for AGNEWS-glove.vec exists
*** load AGNEWS-glove.vec from cache cost 0.042 seconds



{'accuracy': 0.8999332888592395, 'loss': 0.3075038285014477}


In [91]:
for ele in pairs[:30]:
    for ele2 in pairs2:
        if ele[0] == ele2[0]:
            print(ele[0], round(ele[1], 2), round(ele2[1], 2))

tickers 0.0 0.01
Forbes.com 0.0 0.0
peripherals 0.0 0.04
camcorder 0.0 0.09
fax 0.0 0.07
lender 0.0 0.07
KLM 0.0 0.01
WMT.N 0.0 0.01
lifecycle 0.0 0.01
G20 0.0 0.01
securities 0.0 0.08
interface 0.0 0.02
reinsurance 0.0 0.03
quickinfo 0.0 0.0
investing 0.0 0.05
wireless 0.0 0.03
router 0.0 0.03
UAIR.O 0.0 0.01
retirees 0.0 0.07
generates 0.0 0.19
www.investor.reuters.com 0.0 0.01
55-a 0.0 0.01
mortgage 0.0 0.05
inflationary 0.0 0.03
companys 0.0 0.02
ConocoPhillips 0.0 0.01
subscriptions 0.0 0.04
bookkeeping 0.0 0.06
spinner 0.0 0.08
subsystem 0.0 0.01


In [92]:
print(list(map(lambda x: x[0], pairs[:30])))
print(list(map(lambda x: x[0], pairs2[:30])))

['tickers', 'Forbes.com', 'peripherals', 'camcorder', 'fax', 'lender', 'KLM', 'WMT.N', 'lifecycle', 'G20', 'securities', 'interface', 'reinsurance', 'quickinfo', 'investing', 'wireless', 'router', 'UAIR.O', 'retirees', 'generates', 'www.investor.reuters.com', '55-a', 'mortgage', 'inflationary', 'companys', 'ConocoPhillips', 'subscriptions', 'bookkeeping', 'spinner', 'subsystem']
['HealthDay', 'swappers?', 'freeware', '700th', 'NEERGAARD', 'humphead', '8482', 'iPod', 'EMMA', 'LAURAN', "Cash'n'Carrion", 'RFID', 'Conservation', 'Unix', 'handhelds', 'Playstation', 'shareware', 'fourball', 'Gamers', 'Basketball', 'NHL', 'configurator', 'X-43A', 'Forrester', 'Jaschan', 'Shaquille', 'Email', 'Kobe', 'Vitali', 'aggregator']


In [30]:
toy_data = load_data("TOY", "spacy")
vocab = toy_data['vocab']

cache for TOY-spacy.data exists
*** load TOY-spacy.data from cache cost 4.25 seconds


In [3]:
from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

Extracting /tmp/ENZYMES/ENZYMES/ENZYMES.zip
Processing...
Done!


In [30]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')

Processing...
Done!


In [41]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        print(dataset.num_node_features)
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        print(x, x.size())
        print(edge_index, edge_index.size())

        x = self.conv1(x, edge_index)
        print(x.size())
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [44]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(1):
    optimizer.zero_grad()
    out = model(data)
    print(data.train_mask.size())
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

1433
tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0') torch.Size([2708, 1433])
tensor([[   0,    0,    0,  ..., 2707, 2707, 2707],
        [ 633, 1862, 2582,  ...,  598, 1473, 2706]], device='cuda:0') torch.Size([2, 10556])
torch.Size([2708, 16])
torch.Size([2708])


In [35]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float (pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct / data.test_mask.sum().item()
print('Accuracy: {:.4f}'.format(acc))

Accuracy: 0.8040


In [24]:
import torch

x = torch.rand(16, 32, 10, 100)

rand = torch.randint(0, 10, (16, 32))


In [32]:
rsz = rand.size()
x.view(16 * 32, 10, 100).size()

torch.Size([512, 10, 100])